<a href="https://colab.research.google.com/github/Jamie-Huang/NBA-Prediction-Model/blob/main/NBA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bs4
!pip install urlopen
from urllib.request import urlopen
from google.colab import files
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from keras.utils.vis_utils import plot_model
from google.colab import drive 
import tensorflow as tf
from tensorflow import keras
import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate, Concatenate, Dense, LSTM, Input
from tensorflow.keras.metrics import AUC
from keras.models import Sequential
from keras.models import Functional
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
drive.mount('/content/drive')

  Created wheel for urlopen: filename=urlopen-1.0.0-py3-none-any.whl size=1409 sha256=f01bdd6d1459d14cfd57b1deef5180343c4d22ae825c74945bf67b425717d147
  Stored in directory: /root/.cache/pip/wheels/03/ef/31/96a0d9f1487a5af8b393e88ee04994b20ba0b988c922e3aabd
Successfully built urlopen
Mounted at /content/drive


Load in ELO dataframe, change column types

In [ ]:
elo_df = pd.read_csv('/content/game_before_elo_df(full).csv')
elo_df = elo_df.astype({"Year": int, 'Month':int, 'Day':int})
elo_df = elo_df.iloc[:,1:]

cs = elo_df.columns[:30]

main_elo_df = pd.DataFrame(columns = ['Elo', 'Team','Year','Month','Day'])

for c in cs:
  elo_df[c] = elo_df[c].astype('float')
for col in elo_df.columns[:30]:
  temp_df = elo_df[col]
  temp_df = pd.DataFrame(temp_df)
  temp_df['Team'] = col
  temp_df.rename(columns={col: 'Elo',}, inplace=True)
  temp_df['Year'] = elo_df['Year']
  temp_df['Month'] = elo_df['Month']
  temp_df['Day'] = elo_df['Day']
  main_elo_df = main_elo_df.append(temp_df)
main_elo_df.Team = main_elo_df.Team.astype(str)
main_elo_df.Year = main_elo_df.Year.astype('int64')
main_elo_df.Month = main_elo_df.Month.astype('int64')
main_elo_df.Day = main_elo_df.Day.astype('int64')

Joining BoxScores with the ELO rating of each team, downsmapling data to have balanced dataset of wins and losses, hot encode wins and losses

In [ ]:
df=pd.read_csv('/content/scores.csv')
#df = df.reindex(np.random.permutation(df.index))
df = df.iloc[:,1:]
df = df.iloc[:,:-8]
df = df.drop('Home Win',axis=1)
df['Home W'] = np.where(df['HT Score'] > df['AT Score'], 1, 0)
teams = df['AT Name'].unique().tolist()
team_mapping = {'Atlanta Hawks': 'ATL', 'Brooklyn Nets': 'BRK', 'Boston Celtics' : 'BOS', 'Charlotte Bobcats':'CHA','Charlotte Hornets': 'CHH', 'Chicago Bulls': 'CHI', 
                'Cleveland Cavaliers': 'CLE', 'Dallas Mavericks': 'DAL', 'Denver Nuggets': 'DEN', 'Detroit Pistons': 'DET', 'Golden State Warriors': 'GSW',
                'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND', 'Los Angeles Clippers': 'LAC', 'Los Angeles Lakers': 'LAL', 'Memphis Grizzlies': 'MEM',
                'Miami Heat': 'MIA', 'Milwaukee Bucks': 'MIL', 'Minnesota Timberwolves': 'MIN', 'New Orleans Hornets': 'NOH', 'New Orleans Pelicans': 'NOP', 'New Jersey Nets': 'BRK',
                'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC', 'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO', 
                'Portland Trail Blazers': 'POR','Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS', 'Seattle SuperSonics': 'OKC', 'Toronto Raptors': 'TOR',
                'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS'}
df['HT Abbrev'] = df['HT Name'].map(team_mapping)
df['AT Abbrev'] = df['AT Name'].map(team_mapping)
df = df.reset_index()
df = df.rename(columns={"index":"New_ID"})
df['New_ID'] = df.index + 1
df.Year = df.Year.astype('int64')
df.Month = df.Month.astype('int64')
df.Day = df.Day.astype('int64')


df = pd.merge(df,main_elo_df,left_on= ['HT Abbrev','Year','Month','Day'],right_on = ['Team','Year','Month','Day'])
df = df.iloc[:,:-1]
df.columns = [*df.columns[:-1], 'HT Elo']

df = pd.merge(df,main_elo_df,left_on= ['AT Abbrev','Year','Month','Day'],right_on = ['Team','Year','Month','Day'])
main_df = df
df = df.iloc[:,:-1]
df.columns = [*df.columns[:-1], 'AT Elo']
home_w = df[df['Home W'] == 1].iloc[:608,:]
away_w = df[df['Home W'] == 0]
away_w = away_w.append(home_w)
df = away_w
df = df.reindex(np.random.permutation(df.index))

output_df = df['Home W']
test_df = output_df

output_df = output_df.values.flatten()
input_df = df.iloc[:,9:]
input_df = input_df.drop('AT W/L Percentage',axis=1)
input_df = input_df.drop('HT W/L Percentage',axis=1)
input_df = input_df.drop('PS HT W/L Percentage',axis=1)
testing_df = input_df
input_df = input_df.drop('HT Abbrev',axis=1)
input_df = input_df.drop('AT Abbrev',axis=1)
input_df = input_df.drop('Home W',axis=1)
y_train = tf.one_hot(output_df, depth=1,dtype='int8')
input_df.fillna(0,inplace=True)

Rescaling all data to values between 0 and 1

In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))
rescaled_input = scaler.fit_transform(input_df)

In [7]:
rescaled_input = pd.DataFrame(data=rescaled_input, columns=input_df.columns)

In [8]:
l = list(input_df.columns)
home_features = l[2:148] + [l[-2]]
home_input_df = input_df[home_features]

away_features = l[0:2] + l[148:-2] + [l[-1]]
away_input_df = input_df[away_features]

Model with 2 inputs (for home and away teams)

In [ ]:
ht_input = keras.Input(
    shape=(147,), name="Home Team"
) 

dense = layers.Dense(200, activation="relu")
connect_ht = dense(ht_input)

outputs_ht = layers.Dense(100,activation='relu')(connect_ht)

at_input = keras.Input(
    shape=(147,), name="Away Team"
) 

dense = layers.Dense(200, activation="relu")
connect_at = dense(at_input)



outputs_at = layers.Dense(100,activation="relu")(connect_at)

model_ht = keras.Model(inputs=ht_input, outputs=outputs_ht, name="model_ht")
model_at = keras.Model(inputs=at_input, outputs=outputs_at, name='model_at')

combinedInput = concatenate([model_ht.output, model_at.output])

layer4 = layers.Dense(25,activation= 'relu')(combinedInput)
output_layer = layers.Dense(1,activation= 'sigmoid')(layer4)

model = keras.Model(inputs =[model_ht.input,model_at.input],outputs= output_layer)

opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="binary_crossentropy", optimizer='Adam', metrics= ['accuracy'])
history = model.fit(x=[home_input_df,away_input_df], y = y_train, epochs= 200, batch_size=370,validation_split=0.2,shuffle=True)


In [30]:
model = tf.keras.Sequential()
model.add(Dense(294, input_dim=294, activation= 'relu'))
model.add(Dense(300,activation= 'relu'))
model.add(Dense(110,activation= 'relu'))
model.add(Dense(25,activation= 'relu'))
model.add(Dense(1, activation= 'sigmoid'))

In [31]:
model.compile(loss='binary_crossentropy', optimizer= 'Adam' , metrics=['accuracy',AUC(num_thresholds=400)])

In [ ]:
history = model.fit(rescaled_input, y_train, epochs= 200, batch_size=37,validation_split=0.2,shuffle=True)

In [ ]:
loss_train = history.history['accuracy']

loss_val = history.history['val_accuracy']

epochs = range(200)

plt.plot(epochs, loss_train, 'g', label='Training Accuracy')

plt.plot(epochs, loss_val, 'b', label='Validation Accuracy')

plt.title('Training and Validation Accuracy')

plt.xlabel('Epochs')

plt.ylabel('Accuracy')

plt.legend()

plt.show()
